In [ ]:
"Работа с моделью ниже"
%pip install pyarrow==14.0.1  # Версия, совместимая с cudf-cu12 и ibis-framework
%pip install requests==2.31.0  # Версия, требуемая google-colab
%pip install nemo_toolkit['all']  # Попытка установить NeMo

In [2]:
"Скачиваю данные"

# Import necessary library
import os

# Define name of dataset to download
dataset_name = 'Yappy'

# Define Google Drive link to download dataset
download_link = 'https://drive.google.com/uc?id=1wrXHqu1c_f54r-d9Bz5DEBq3Fk3tr-4_'

# Define path to save downloaded zip file
zip_file_path = f'{dataset_name}.zip'


# Download dataset zip file and unzip it
os.system(f'gdown -q {download_link} -O {zip_file_path}')
os.system(f'unzip -q -n {zip_file_path}')

# Verify successful download and extraction of dataset
if os.path.exists(zip_file_path):
    print('\n\033[1m\033[32m' + f'Download and unzip of {dataset_name} successful' + '\033[0m\n')



Download and unzip of Yappy successful



In [3]:
"Загружаю ссылки в датафрейм"
import pandas as pd
df = pd.read_csv('15. Yappy/yappy_hackaton_2024_400k.csv')
pd.set_option('display.max_colwidth', None)
df.head()

,link,description
0,https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin"
1,https://cdn-st.rutubelist.ru/media/39/6c/b31bc6864bef9d8a96814f1822ca/fhd.mp4,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #roblox #роблокс
2,https://cdn-st.rutubelist.ru/media/e9/e0/b47a9df14a5e97942715e5e705c0/fhd.mp4,"#boobs , #красивыедевушки , #ass"
3,https://cdn-st.rutubelist.ru/media/87/43/b11df3f344d0af773aac81e410ee/fhd.mp4,NaN
4,https://cdn-st.rutubelist.ru/media/d1/e7/642dc2194fcdb69664f832d5f2dd/fhd.mp4,NaN


In [35]:
"-------------Удаляю папки (Опционально)----------------"
import os
import shutil

def delete_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f'Папка {folder_path} и все ее содержимое были успешно удалены.')
    else:
        print(f'Папка {folder_path} не найдена')
delete_folder('Videos')
delete_folder('Audio')

Папка Videos не найдена
Папка Audio и все ее содержимое были успешно удалены.


In [38]:
"-------------Создаю папки для видосов----------------"

import requests
import os
# Создать папки для изображений, если они еще не созданы
if not os.path.exists('Videos'):
    os.makedirs('Videos')

In [39]:
"Ограничение на кол-во видосов и папка для сохранения. ПОМЕНЯТЬ КОЛ-ВО ВИДОСОВ В КОНЦЕ"

import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd

output_folder = 'Videos'
df_vid = df.iloc[10000:15000].copy()  # Предполагаем, что df уже определен

In [40]:
"Многопоточное скачивание видосов"

def download_video(url, output_folder):
    local_filename = url.split('/')[-2] + '.mp4'  # Добавляем расширение .mp4 к имени файла
    path = os.path.join(output_folder, local_filename)  # Полный путь к файлу
    if os.path.exists(path):
        print(f"File {local_filename} already exists. Skipping download.")
        return path
    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        return path
    except requests.RequestException as e:
        print(f"Failed to download {url}. Error: {e}")
        return None

# Использование ThreadPoolExecutor для многопоточной загрузки
results = []
with ThreadPoolExecutor(max_workers=20) as executor:  # Определите подходящее количество потоков
    futures = {executor.submit(download_video, url, output_folder): url for url in df_vid['link']}
    # Использование tqdm для отображения прогресса загрузки
    for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading videos"):
        url = futures[future]
        try:
            path = future.result()
            results.append((url, path))
        except Exception as e:
            print(f"Video download failed for {url}. Error: {e}")

# Обновление DataFrame с путями к скачанным файлам
for url, path in results:
    df_vid.loc[df_vid['link'] == url, 'download_path'] = path

File 8b992fd14678b870f800acbcbaa8.mp4 already exists. Skipping download.
File 6eb762ed4cca90873404b4a134ba.mp4 already exists. Skipping download.
File 4834ae9d447db7a69a552fac87a9.mp4 already exists. Skipping download.
File 01469823490398eb770f1973c8ff.mp4 already exists. Skipping download.
File 812088634d3581c2d4598734c9e3.mp4 already exists. Skipping download.
File 56df38504587bce2e3e4fbaf337c.mp4 already exists. Skipping download.


File 673435e64ae0b6a9528ec4c94406.mp4 already exists. Skipping download.
File 80b78eb147a890ac95e1d37d06bc.mp4 already exists. Skipping download.
File 8736b18940c5ac883578fc0de475.mp4 already exists. Skipping download.
File 5d21f4db4127b517512f10e00c75.mp4 already exists. Skipping download.
File 9d2c06574f5b825db2f15dd024b2.mp4 already exists. Skipping download.
File fd7e99f54e0087d8cd670cfa395c.mp4 already exists. Skipping download.
File 67de78974601a5e3dee07f173838.mp4 already exists. Skipping download.
File a738857f4dc28bfe7dc6eb039560.mp4 already exists. Skipping download.
File 46cdedf64931b9f19771e3c53903.mp4 already exists. Skipping download.
File 40d82fb84cd680b1651fa941664f.mp4 already exists. Skipping download.
File f52ebf2741979885617de95d815e.mp4 already exists. Skipping download.
File 2ff5aa1f4bcfbbdc1d0a2bb2e712.mp4 already exists. Skipping download.
File 1fa833fd4f41bf210c1942e77ac6.mp4 already exists. Skipping download.
File 67dec4df4387be6123fd54322d93.mp4 already exist

File cb0c0e6c4fb294238c79e841e800.mp4 already exists. Skipping download.
File 3130ff774419b974af730f333372.mp4 already exists. Skipping download.
File 318c52794792b40aef531d832ed9.mp4 already exists. Skipping download.
File 047cc97e42d29ebd35c9630a2011.mp4 already exists. Skipping download.
File da32db1d4906b073fc0404b7e9bc.mp4 already exists. Skipping download.
File 118ddcfc45279b73a5953b98d580.mp4 already exists. Skipping download.
File ee22a1da4bb8b7be5c4e55483cf8.mp4 already exists. Skipping download.
File 6ce8f4f841c4a5d201cae4d8563a.mp4 already exists. Skipping download.
File c25fd7ac474182b47bfb552a3f55.mp4 already exists. Skipping download.
File 6fdb5a41474bbf33ea7d318c80af.mp4 already exists. Skipping download.
File da97484340a4b468ffdd6e6908ad.mp4 already exists. Skipping download.
File 18f4fbd940cd9bab0a6ceb48498a.mp4 already exists. Skipping download.
File b601b8064bc79eccb82ec77adb6f.mp4 already exists. Skipping download.
File 916aa89940d09e2e89840a222040.mp4 already exist

File 05a0cc5f4034a133bfff69589841.mp4 already exists. Skipping download.


In [41]:
"Папка для аудио"
if not os.path.exists('Audio'):
    os.makedirs('Audio')
    print('Папка Audio создана')
else:
  print('Папка Audio уже существует')

Папка Audio создана


In [42]:
"Достанем аудио из видео"
from moviepy.editor import *
import logging
# logging.getLogger('moviepy').setLevel(logging.CRITICAL)
from moviepy.editor import VideoFileClip
import os

def extract_audio(video_url):
    try:
        video = VideoFileClip(video_url)
        audio = video.audio
        audio_file_name = os.path.basename(video_url).split('.')[0] + '.wav'
        audio_file_path = os.path.join('Audio', audio_file_name)
        audio.write_audiofile(audio_file_path, fps=16000, verbose=False, logger=None)
        audio.close()
        video.close()

        os.remove(video_url)

        return audio_file_path
    except Exception as e:
        print(f"Error processing {video_url}: {e}")
        return None

In [ ]:
"Достанем аудио из видео"
from joblib import Parallel, delayed
from tqdm.contrib.concurrent import process_map
print('Начинаю извлекать аудиодорожки из видео. Они будут в папке Audio')
df_vid['audio_file'] = process_map(extract_audio, df_vid['download_path'], max_workers=os.cpu_count(), chunksize=20)
print('Извлечение аудиодорожек завершено')

In [44]:
import soundfile as sf
import librosa
import tempfile
import nemo.collections.asr as nemo_asr
from tqdm.auto import tqdm
import torch

In [ ]:
"ВАРИАНТ С CUDA ТОЛЬКО ПРОБНЫЙ. ПРОПУСТИТЬ ИЛИ ПОТЕСТИТЬ"

# Загрузка предобученной модели
asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="nvidia/stt_ru_fastconformer_hybrid_large_pc")

# Перенос модели на CUDA
if torch.cuda.is_available():
    asr_model = asr_model.to('cuda')
else:
    print("CUDA is not available. Please check your installation.")


In [ ]:
"ВАРИАНТ С CUDA ТОЛЬКО ПРОБНЫЙ. ПРОПУСТИТЬ ИЛИ ПОТЕСТИТЬ"

import torch
import nemo.collections.asr as nemo_asr
import librosa
import soundfile as sf
import tempfile
from tqdm import tqdm

# Загрузка предобученной модели
asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="nvidia/stt_ru_fastconformer_hybrid_large_pc")

# Перенос модели на CUDA
if torch.cuda.is_available():
    asr_model = asr_model.to('cuda')
else:
    print("CUDA is not available. Please check your installation.")

# Функция для распознавания речи
def speech_recognition_nemo(audio_file_path):
    try:
        # Загрузка аудиофайла и преобразование в моно
        audio, rate = librosa.load(audio_file_path, sr=16000, mono=True)
        # Преобразование аудио в тензор PyTorch и перенос на CUDA
        audio_tensor = torch.tensor(audio).unsqueeze(0)  # Добавляем размерность батча
        if torch.cuda.is_available():
            audio_tensor = audio_tensor.to('cuda')
    except Exception as e:
        print(f"Ошибка загрузки аудио: {e}. Файл: {audio_file_path}")
        return None

    try:
        # Создание временного файла для передачи в NeMo
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as temp_audio_file:
            sf.write(temp_audio_file.name, audio_tensor.cpu().numpy(), rate)
            temp_audio_file.seek(0)  # Возвращаемся к началу файла перед чтением
            transcription = asr_model.transcribe([temp_audio_file.name])[0]
    except Exception as e:
        print(f"Ошибка распознавания речи: {e}. Файл: {audio_file_path}")
        return None
    return transcription

# Применение модели ко всем аудио файлам в DataFrame
df_vid['ASR_NEMO_BETTER'] = [speech_recognition_nemo(file_path) for file_path in tqdm(df_vid['audio_file'], desc="Applying NeMo ASR model")]

In [45]:
asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="nvidia/stt_ru_fastconformer_hybrid_large_pc")

[NeMo I 2024-06-16 08:21:09 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-06-16 08:21:10 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - - /data/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    is_tarred: true
    tarred_audio_filepaths:
    - - /data/audio__OP_0..1023_CL_.tar
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    is_concat: false
    concat_sampling_technique: random
    concat_sampling_probabilities: ''
    
[NeMo W 2024-06-16 08:21:10 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the

[NeMo I 2024-06-16 08:21:10 features:289] PADDING: 0


[NeMo W 2024-06-16 08:21:11 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2024-06-16 08:21:12 rnnt_models:217] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-06-16 08:21:13 save_restore_connector:249] Model EncDecHybridRNNTCTCBPEModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_ru_fastconformer_hybrid_large_pc/snapshots/9d65ebcc12bedae82b23ceadae4e8abb9fbd27a0/stt_ru_fastconformer_hybrid_large_pc.nemo.


In [46]:
# Предположим, что asr_model уже загружена и готова к использованию
def speech_recognition_nemo(audio_file_path):
    try:
        # Загрузка аудиофайла и преобразование в моно
        audio, rate = librosa.load(audio_file_path, sr=16000, mono=True)
    except Exception as e:
        print(f"Ошибка загрузки аудио: {e}. Файл: {audio_file_path}")
        return None  # Возвращаем None, если загрузить файл не удалось

    try:
        # Создание временного файла для передачи в NeMo
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as temp_audio_file:
            sf.write(temp_audio_file.name, audio, rate)
            temp_audio_file.seek(0)  # Возвращаемся к началу файла перед чтением
            transcription = asr_model.transcribe([temp_audio_file.name])[0]
    except Exception as e:
        print(f"Ошибка распознавания речи: {e}. Файл: {audio_file_path}")
        return None  # Возвращаем None, если произошла ошибка при распознавании
    return transcription

In [ ]:
df_vid['ASR_NEMO_BETTER'] = [speech_recognition_nemo(file_path) for file_path in tqdm(df_vid['audio_file'], desc="Applying NeMo ASR model")]

In [26]:
df_vid

,link,description,download_path,audio_file,ASR_NEMO_BETTER
0,https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin",Videos/ef285e0241139fc611318ed33071.mp4,Audio/ef285e0241139fc611318ed33071.wav,[]
1,https://cdn-st.rutubelist.ru/media/39/6c/b31bc6864bef9d8a96814f1822ca/fhd.mp4,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #roblox #роблокс,Videos/b31bc6864bef9d8a96814f1822ca.mp4,Audio/b31bc6864bef9d8a96814f1822ca.wav,"[Сейчас я тебе покажу секретную команду в Роблоксе. Только ее активировать поставь лайк отпишись, а также веди в чат команду ай лов ю, когда вы ее ведете, у вас на экране появится вот такой клоун, Скрипт был создан одним из создателей Роблокса и работает только в играх. Где есть Робласти Эвлоп сервис, чтобы выбрать скример для всего-то выйти из игры, но лучше не проверяли.]"
2,https://cdn-st.rutubelist.ru/media/e9/e0/b47a9df14a5e97942715e5e705c0/fhd.mp4,"#boobs , #красивыедевушки , #ass",Videos/b47a9df14a5e97942715e5e705c0.mp4,Audio/b47a9df14a5e97942715e5e705c0.wav,[]
3,https://cdn-st.rutubelist.ru/media/87/43/b11df3f344d0af773aac81e410ee/fhd.mp4,NaN,Videos/b11df3f344d0af773aac81e410ee.mp4,Audio/b11df3f344d0af773aac81e410ee.wav,"[что он о тебе думает прямо сейчас. Слушай, думает, что ты знаешь, для него непосильная ноша, как будто бы даролева мечеи. Чувствует то, что ты по характеру гораздо сильнее и то, что ему с тобой будет очень сложно, но при этом есть желание и желания не совсем приличное. Проявляться боится и честно говоря, сам проявляться не будет. Я бы на твоем месте вообще подумала. Нужен ли тебе такой партнер, который слабее тебя? Ну, выводы делай сама.]"
4,https://cdn-st.rutubelist.ru/media/d1/e7/642dc2194fcdb69664f832d5f2dd/fhd.mp4,NaN,Videos/642dc2194fcdb69664f832d5f2dd.mp4,Audio/642dc2194fcdb69664f832d5f2dd.wav,"[Я убежден, что чем беднее человек, тем сложнее его удовлетворить, потому что тот человек, у которого нет денег, он всегда недоволен чем-то он всегда просит больше, тот человек, который, в принципе, удовлетворен своими деньгами, он говорит классно. Это здорово Спасибо, и идет и делает.]"
...,...,...,...,...,...
95,https://cdn-st.rutubelist.ru/media/6a/29/c0c860684a0e9c0e23d496830525/fhd.mp4,#красивыедевушки #танцы,Videos/c0c860684a0e9c0e23d496830525.mp4,Audio/c0c860684a0e9c0e23d496830525.wav,[]
96,https://cdn-st.rutubelist.ru/media/e9/88/481fdc674d0b8161dcc7426a49eb/fhd.mp4,#красивыедевушки #попа #ass,Videos/481fdc674d0b8161dcc7426a49eb.mp4,Audio/481fdc674d0b8161dcc7426a49eb.wav,"[меня, люби, Жаки маня, ночу.]"
97,https://cdn-st.rutubelist.ru/media/27/65/39b479d1466891d42aa335bd1c18/fhd.mp4,#путешествия #journey #туризм #море #красивыеместа #интересныелокации,Videos/39b479d1466891d42aa335bd1c18.mp4,Audio/39b479d1466891d42aa335bd1c18.wav,"[Как прошел мой год? Ну, как вам сказать бай рубает, скетчу листом стайвает столь бесно?]"
98,https://cdn-st.rutubelist.ru/media/d1/0f/f41e4b0d40ccb994e792920e3566/fhd.mp4,NaN,Videos/f41e4b0d40ccb994e792920e3566.mp4,Audio/f41e4b0d40ccb994e792920e3566.wav,"[Посмотрим на гигантские царапины ближе, а туда сейчас спустимся и все увидим, погнали.]"


In [15]:
"Поменяй название"
df_vid.to_pickle('asr_10-20k.pkl')